<a href="https://colab.research.google.com/github/Billy-Drunkenstein/MAFN/blob/main/Spring%202025/Machine%20Learning%20for%20Finance/Homework%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Context of the problem**

The problem centers around trying to predict S&P 500 stock market movements using:

Volatility Index, which measures market sentiment and expected volatility

Crude Oil Futures, which can serve as a proxy for global economic activity.

Gold Futures, often used as a safe-haven asset during times of market stress.

The respective tickers for each of these items are the following:

^GSPC = SP500

^VIX = Volatility index

CL=F = Crude oil futures

GC=F = Gold futures

In [1]:
Tickers = {
    '^GSPC': 'S&P 500',
    '^VIX': 'Vol Index',
    'CL=F': 'Crude Oil Futures',
    'GC=F': 'Gold Futures',
}

**(0 points) Prerequisite - run the code to import the necessary modules**

In [25]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime

from sqlalchemy import create_engine
#Creates a connection (engine) to an SQLite database.

import sqlite3
#sqlite3 is imported as well, although in this code the SQLAlchemy engine is used to handle database I/O

from sklearn.metrics import accuracy_score, classification_report, log_loss
#sklearn.metrics:Provides functions such as accuracy_score and classification_report to evaluate model performance

from sklearn.linear_model import LogisticRegression, SGDClassifier
#	•	sklearn.linear_model: Provides linear models: LogisticRegression is used to fit classification models with various regularization methods. SGDClassifier is a stochastic gradient descent classifier that can be trained incrementally (allowing for iteration callbacks).

from sklearn.neighbors import KNeighborsClassifier
#KNeighborsClassifier (for k-nearest neighbors

from sklearn.svm import SVC
#SVC (for support vector machines)

from sklearn.tree import DecisionTreeClassifier
#DecisionTreeClassifier (for decision trees)

from sklearn.ensemble import RandomForestClassifier
#RandomForestClassifier (for random forests)

**(0 points)Prerequisite - how to download data from Yahoo Finance without having a premium subscription**

*Option 1: Excel*

1) Go in browser to finance.yahoo.com put stock ticker in a box


2) On the left select HISTORICAL DATA


3) For period select MAX


4) SELECT ALL from table then COPY


5) Open new sheet of excel in excel select PASTE AS HTML


6) Delete any extra data from Excel


7) Import the data from the Excel file

*Option 2: Yahoo Finance workaround*

Run this python code to download the data.

!pip install yfinance openpyxl

-------------------------

msft_data = yf.download("MSFT", start="2014-01-24", end="2025-01-15", interval="1d", auto_adjust=False)

if "Adj Close" not in msft_data.columns and "Adj Close" in msft_data.columns.str.lower():
    msft_data.rename(columns={"adj close": "Adj Close"}, inplace=True)

file_path = "MSFT_Historical_Data.xlsx"
msft_data.to_excel(file_path, engine="openpyxl")

**Task 1 - Data preparation and exploration**



(10 points) Task 1.1 - Create a function to download data via API. Hint: Data will be downloaded from Yahoo Finance

In [3]:
def download_data(tickers, start_date, end_date):
    """
    Downloads historical 'Adj Close' or 'Close' data for the given tickers from Yahoo Finance.
    Handles cases where 'Adj Close' is missing.
    """
    data_dict = {}

    for ticker in tickers:
        data = yf.download(ticker, start = start_date, end = end_date,
                           interval = '1d', auto_adjust = False, progress = False)

        # Keep only the 'Adj Close' column
        if 'Adj Close' in data.columns:
            price = data['Adj Close'].rename(columns = {'Adj Close' : ticker})
            print(ticker, 'Adj Close')

        else:
            price = data['Close'].rename(columns = {'Close' : ticker})
            print(ticker, 'Close')

        # Rename Column
        data_dict[ticker] = price

    # Assemble DataFrame
    price_df = pd.concat(data_dict.values(), axis = 1, join = 'inner')
    price_df.index.name = None
    price_df.columns.name = None

    return price_df

In [4]:
start = '2014-01-24'
end = '2025-01-15'

Data = download_data(Tickers, start, end)

^GSPC Adj Close
^VIX Adj Close
CL=F Adj Close
GC=F Adj Close


(10 points) Task 1.2 - Create a function to clean the data. Hint: Drop the rows with missing values.

Calculate the daily percentage change (return) for the S&P 500. Create a binary target variable: 1 if the next day's S&P 500 return > 0, else 0.


In [5]:
def clean_data(df):
    """
    Drops rows with missing NA
    Calculates daily return
    Creates binary target variable
    """

    df.dropna(inplace = True)

    # Daily return
    df['Return'] = df['^GSPC'].pct_change()

    # Boolean target
    df['Target'] = (df['Return'] > 0).shift(-1)

    # Drop final row
    df.dropna(inplace = True)

    return df

In [6]:
Data = clean_data(Data)

print(Data.isna().sum().sum())
Data.head()

0


,^GSPC,^VIX,CL=F,GC=F,Return,Target
2014-01-27,1781.560059,17.420000,95.720001,1263.599976,-0.004876,True
2014-01-28,1792.500000,15.800000,97.410004,1251.000000,0.006141,False
2014-01-29,1774.199951,17.350000,97.360001,1262.199951,-0.010209,True
2014-01-30,1794.189941,17.290001,98.230003,1242.199951,0.011267,False
2014-01-31,1782.589966,18.410000,97.489998,1240.099976,-0.006465,False


(10 points) Task 1.3 - Create a function to save the data to a local database

In [7]:
def save_to_db(df, db_name='market_data.db', table_name='market_data'):
    """
    Saves a DataFrame to a local database
    """

    # Create connection to SQL database
    conn = sqlite3.connect(db_name)

    # Save the DataFrame to the database
    df.to_sql(table_name, conn, if_exists = "replace", index = True)

    conn.close()


def read_from_db(db_name='market_data.db', table_name='market_data'):
    conn = sqlite3.connect(db_name)
    df = pd.read_sql(f"SELECT * FROM {table_name}", conn)
    conn.close()

    df.set_index("index", inplace=True)
    df.index.name = None
    df.index = pd.to_datetime(df.index).date
    return df

In [8]:
save_to_db(Data)

temp = read_from_db()
temp.head()

,^GSPC,^VIX,CL=F,GC=F,Return,Target
2014-01-27,1781.560059,17.420000,95.720001,1263.599976,-0.004876,1
2014-01-28,1792.500000,15.800000,97.410004,1251.000000,0.006141,0
2014-01-29,1774.199951,17.350000,97.360001,1262.199951,-0.010209,1
2014-01-30,1794.189941,17.290001,98.230003,1242.199951,0.011267,0
2014-01-31,1782.589966,18.410000,97.489998,1240.099976,-0.006465,0


**(10 points) Task 2 – Create a function to split the Data into Train, Test, and Validation Sets**. Hint: Ensure the data is sorted by date first.

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def split_data(df, feature_columns, target_column):
    """
    Splits input df into training, validation, and testing sets
    Scales feature columns by Standard Scaler fit on training set
    """

    # Sort by date
    df = df.sort_index()

    # Select features and target
    X = df[feature_columns]
    y = df[target_column].astype(int)

    # Train test split
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

    # Train val split
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size  =0.2, shuffle = False)

    # Normalize feature columns
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    return X_train, y_train, X_val, y_val, X_test, y_test

**(10 points) Task 3.1 - Create a function to analyse the data applying loss functions and regularisation functions such as ridge and lasso regulsatisation**

In [28]:
def logistic_regression_models(X_train, y_train, X_val, y_val, max_iter = 1000):
    """
    Evaluates Logistic Regression quality on a given train-eval set
    Using Ridge (L2) and Lasso (L1) regularization

    Assumes: X_train and X_val are normalized
    """

    # Result matrix
    results = {}

    # Ridge
    # Default max_iter at 100 fails to converge
    # Change to 1000 iterations
    ridge_model = LogisticRegression(penalty="l2", max_iter=max_iter, class_weight = 'balanced')
    ridge_model.fit(X_train, y_train)

    ridge_preds = ridge_model.predict(X_val)
    ridge_accuracy = accuracy_score(y_val, ridge_preds)
    ridge_report = classification_report(y_val, ridge_preds, output_dict=True, zero_division = 0)

    ridge_probs = ridge_model.predict_proba(X_val)
    ridge_logloss = log_loss(y_val, ridge_probs)

    results["Ridge (L2)"] = {
        "Accuracy": ridge_accuracy,
        "Precision": ridge_report["weighted avg"]["precision"],
        "Recall": ridge_report["weighted avg"]["recall"],
        "F1-Score": ridge_report["weighted avg"]["f1-score"],
        "Log Loss": ridge_logloss
    }

    # Lasso
    # lbfgs solver does not support l1
    # Switch to liblinear instead
    lasso_model = LogisticRegression(penalty="l1", solver="liblinear", max_iter=max_iter, class_weight = 'balanced')
    lasso_model.fit(X_train, y_train)

    lasso_preds = lasso_model.predict(X_val)
    lasso_accuracy = accuracy_score(y_val, lasso_preds)
    lasso_report = classification_report(y_val, lasso_preds, output_dict=True, zero_division = 0)

    lasso_probs = lasso_model.predict_proba(X_val)
    lasso_logloss = log_loss(y_val, lasso_probs)

    results["Lasso (L1)"] = {
        "Accuracy": lasso_accuracy,
        "Precision": lasso_report["weighted avg"]["precision"],
        "Recall": lasso_report["weighted avg"]["recall"],
        "F1-Score": lasso_report["weighted avg"]["f1-score"],
        "Log Loss": lasso_logloss
    }

    results = pd.DataFrame.from_dict(results)

    return results

In [29]:
feature_columns = ["^GSPC", "^VIX", "CL=F", "GC=F", "Return"]
target_column = "Target"

X_train, y_train, X_val, y_val, X_test, y_test = split_data(Data, feature_columns, target_column)

results = logistic_regression_models(X_train, y_train, X_val, y_val)

print(results)

           Ridge (L2)  Lasso (L1)
Accuracy     0.503401    0.503401
Precision    0.253413    0.501744
Recall       0.503401    0.503401
F1-Score     0.337119    0.352590
Log Loss     0.703035    0.699651


### The above result is consistent but barely better than random guesses. Ridge's precision is only 0.25, which means its positive return prediction is only 25% of the time correct.
### The F-1 scorres are around 0.35, which is better than some of the financial time series forecast models I tried before. Log loss of around 0.7 is atrocious. I wouldn't base any trading strategies on Logistic Regression

**(15 points) Task 4 - Create an iteration callback function to print information for troubleshooting**

During each epoch, the callback should print:
A. The current epoch number.
B. The loss value for that epoch.
C. The training accuracy for that epoch.

Comment on how the loss evolves over the epochs and whether this trend indicates proper convergence?

In [34]:
def training_callback(epoch, loss, accuracy):
    """
    Prints information for each epoch
    """

    print(f"Epoch {epoch}: Loss = {loss:.4f}, Accuracy = {accuracy:.2%}\n")
    return

def sgd_classifier_with_callback(X_train, y_train, n_epochs=10):
    """
    Evaluates SGDClassifier performance over a training set

    Assumes: X_train is normalized
    """

    model = SGDClassifier(loss="log_loss", warm_start=True, max_iter = 100, eta0=0.01, learning_rate="optimal")

    # Need to be parsed for the first partial fit
    classes = np.unique(y_train)

    # Assumes X_train is already normalized
    # Training loop
    for epoch in range(1, n_epochs + 1):
        # One iteration
        if epoch == 1:
            model.partial_fit(X_train, y_train, classes=classes)
        else:
            model.partial_fit(X_train, y_train)

        y_pred = model.predict(X_train)
        y_prob = model.predict_proba(X_train)

        # Calculate statistics
        loss = log_loss(y_train, y_prob)
        accuracy = accuracy_score(y_train, y_pred)

        # Callback
        training_callback(epoch, loss, accuracy)

    return model

In [35]:
model = sgd_classifier_with_callback(X_train, y_train, 10)

Epoch 1: Loss = 16.2534, Accuracy = 54.91%

Epoch 2: Loss = 16.2534, Accuracy = 54.91%

Epoch 3: Loss = 19.7903, Accuracy = 45.09%

Epoch 4: Loss = 16.2534, Accuracy = 54.91%

Epoch 5: Loss = 18.1752, Accuracy = 49.57%

Epoch 6: Loss = 18.0729, Accuracy = 49.86%

Epoch 7: Loss = 19.7903, Accuracy = 45.09%

Epoch 8: Loss = 19.7903, Accuracy = 45.09%

Epoch 9: Loss = 16.2534, Accuracy = 54.91%

Epoch 10: Loss = 16.2534, Accuracy = 54.91%



**(10 points) Task 5 - Create a function to explore classification algorithms in the following order: Nearest Neighbor,SVM, Decision Trees and Random Forest**.

Using the features from your dataset, your goal is to predict whether the S&P 500’s next-day return will be positive (1) or negative (0).

For each algorithm, compute and report accuracy on the test set and a detailed classification report (including precision, recall, and F1-score).

Hint: This can all be done within a single function, using a for loop.

In [52]:
def explore_classification_algorithms(X_train, y_train, X_test, y_test):
    """
    Evaluate multiple classification algorithms on a given dataset

    Assumes: X_train and X_test are normalized
    """

    models = {
        'Nearest Neighbor': KNeighborsClassifier(),
        'SVM': SVC(probability=True, class_weight="balanced"),
        'Decision Tree': DecisionTreeClassifier(class_weight="balanced"),
        'Random Forest': RandomForestClassifier(class_weight="balanced")
    }

    results = {}

    for name, model in models.items():
        # Fit and Predict
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict = True, zero_division=0)

        y_prob = model.predict_proba(X_test)
        logloss = log_loss(y_test, y_prob)

        # Compile Results
        results[name] = {
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': report['1']['precision'],
            'Recall': report['1']['recall'],
            'F1-Score': report['1']['f1-score'],
            'Log Loss': logloss
        }

    results = pd.DataFrame(results).T

    return results

In [53]:
explore_classification_algorithms(X_train, y_train, X_test, y_test)

,Accuracy,Precision,Recall,F1-Score,Log Loss
Nearest Neighbor,0.463768,0.714286,0.033113,0.063291,0.727941
SVM,0.476449,0.522968,0.490066,0.505983,0.688281
Decision Tree,0.489130,0.537879,0.470199,0.501767,18.413606
Random Forest,0.489130,0.589286,0.218543,0.318841,0.736471


**(15 points) Task 6 - Create a simple data pipeline, which can be a scheduled script.** Hint: This serves as the main entry point that strings together all of the previous functions to form a complete data pipeline. This should be scheduled at regular intervals.

In [ ]:
def run_pipeline():

**(10 points) Task 7 - What does the accuracy of each method used seem to be? What does this tell us both about the data that we used to train our prediction model and the predictive techniques which we tried to implement?**